In [11]:
from natasha import AddressExtractor

In [23]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from tqdm.notebook import tqdm
import time

In [13]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

In [14]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    lower = (word.lower() for word in nltk.wordpunct_tokenize(text))
    pattern = re.compile("[а-я+0-9-]+")
    words = pattern.findall(" ".join(lower))
    norm_form = (morph.parse(word)[0].normal_form for word in words)
    tokens = [word for word in text if word not in stop_words]
    return " ".join(tokens)

In [61]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я+0-9-]+")
    words = pattern.findall(" ".join(tok))
    #norm_form = (stemmer.stem(word.lower()) for word in words)
    tokens = [word for word in words if word not in stop_words]
    return " ".join(tokens)

In [62]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)

In [63]:
df = data[:100]

In [64]:
text = 'Мойка на ПЛАНЕРНОЙ объявляет об АКЦИЯХ :\n\n✨Покрытие Тефлон (защитное покрытие блеск+гидрофоб)\nСедан 1 000 рублей\nКроссовер 1 250 рублей\nДжип 1 500 рублей\n(В стоимость включена мойка авто, обезжиривание, полимерное покрытие с тефлоном)\nАктуально в дождь, т.к. имеет высокий гидрофобный эффект и машина дольше остаётся чистой!!!\n\n✨ защитное гидрофобное покрытие для стёкол\nЛобовое стекло 1 200 рублей\nЛобовое стекло\n+ передние боковые 1 500 рублей\n(Имеет сильный гидрофобный эффект,\nчто обеспечивает комфортное вождение в условиях дождя! Срок службы 6 месяцев! )\n\n✨Мойка ЛЮКС\nСедан/кроссовер 500 руб\nДжип 700 руб\n(В стоимость входит мойка СТАНДАРТ, пылесос салона, протирание всех пластиковых поверхностей, нанесение полиролей для пластика, мойка стёкол )\n\nНа остальные виды работ уточняйте цены по тел: 8-921-750-35-86 или в\nличные сообщения https://vk.com/milamaier\n\n📍Адрес: Санкт-Петербург, ул. Планерная, 24'

In [77]:
f = open('data/street_list_spb(without type).txt', 'r')
l = [line.strip() for line in f]

In [81]:
k = 0
o = []
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    p = []
    for j in l:
        if sample.find(stemmer.stem(j)) > 0:
            p.append(j)
    o.append(p)

In [82]:
d = 0
for i in o:
    if len(i)>0:
        d+=1
print(d)

87


In [15]:
extractor = AddressExtractor()
sample = process_text(text)
matches = extractor(process_text(sample))
for match in matches:
    print(match.fact)

Address(parts=[Settlement(name='Санкт-Петербург', type=None), Street(name='Планерная', type='улица'), Building(number='24', type=None)])


In [37]:
df['text'][8]

'1. Эвакуировали 384 лицей. (в 14:30 всех запустили назад) \n2. Эвакуировали гимназию 405\n3. В 14:00 эвакуировали 98 школу\n4. Эвакуировали гимназию 399\n5. Эвакуировали гимназию 41\n6. 295 гимназию эвакуируют.\n\nЭвакуировали учащихся из школы № 553 Фрунзенского района на улице Ярослава Гашека, 4, корп. 4'

In [38]:
k = 0
f = open('street.txt', 'w')
extractor = AddressExtractor()
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    matches = extractor(sample)
    s = ''
    for match in matches:
        for j in range(0, len(match.fact.parts)):
            try:
                if match.fact.parts[j].type is None:
                    try:
                        s += match.fact.parts[j].number + ' '
                        continue
                    except:
                        s += match.fact.parts[j].name + ' '
                        continue
                try:
                    if match.fact.parts[j].number is None:
                        break
                    s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
                except:
                    if match.fact.parts[j].name is None:
                        continue
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
            except:
                continue
    f.write(s + '\n')
    if len(matches)>0:
        k += 1
f.close()

In [30]:
sample = process_text('Россия Санкт-Петербург Ботаническая улица 66')
matches = extractor(sample)
s = ''
for match in matches:
    for j in range(0, len(match.fact.parts)):
        try:
            if match.fact.parts[j].type is None:
                try:
                    s += match.fact.parts[j].number + ' '
                    continue
                except:
                    s += match.fact.parts[j].name + ' '
                    continue
            try:
                if match.fact.parts[j].number is None:
                    break
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
            except:
                if match.fact.parts[j].name is None:
                    continue
            s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
        except:
            continue

In [31]:
s

'Санкт-Петербург улица Ботаническая 66 '

In [112]:
process_text(text)

'На Комсомола улице дома 10 уехал светофор второй середине перекрестка'

In [124]:
extractor = AddressExtractor()
matches = extractor(process_text(list(df['text'])[0]))
for match in matches:
    print(match.span, match.fact)

In [125]:
len(matches)

0

In [94]:
match.fact.parts

[Street(name='Комсомола',
        type='улица'), Building(number='10',
          type='дом')]

In [183]:
s = match.fact.parts[0].name + ' ' + match.fact.parts[0].type

In [184]:
for i in range(0, len(match.fact.parts)):
    if match.fact.parts[i].type is None:
        s += match.fact.parts[i].number
    else:
        s += match.fact.parts[i].type + ' ' + match.fact.parts[i].number + ' '

TypeError: can only concatenate str (not "NoneType") to str

In [185]:
s

'Лиговском проспект'

In [82]:
match.fact.parts[2].type is None

True

In [86]:
f = open('street.txt', 'r')
l = [line.strip() for line in f]

In [88]:
len(l)

58123

In [89]:
data.shape

(58123, 3)

In [90]:
data['natasha'] = l

In [92]:
data.loc[data['natasha'] != '']

,id,date,text,natasha
8,12341152,2019-12-09 11:18:58,1. Эвакуировали 384 лицей. (в 14:30 всех запус...,район Фрунзенского улица Ярослава Гашека 4 кор...
39,12338074,2019-12-08 15:58:13,"Сегодня в 8 утра, на Кантемировской улице, мо...",улица Кантемировской 9
52,12336390,2019-12-08 06:53:59,Сегодня в 3 часа ночи на Будапештской ул. 95 к...,улица Будапештской 95 корпус 1
53,12336388,2019-12-08 06:52:39,"Туристская, 28.\n5ого декабря с утра обнаружил...",улица Туристкая 26
56,12335402,2019-12-07 19:12:55,Новости города за сегодняшний день:\n\n1. Липо...,набережная Петроградской 6
...,...,...,...,...
58046,4809342,2016-09-11 23:52:04,6 сентября был угнан мой любимый Форд Фокус М7...,бульвар Сиреневом 6
58052,4807823,2016-09-11 19:45:50,На Ропшинском шоссе 08 09.2016\n(Смотреть с 0;...,шоссе На Ропшинском 08
58057,4807684,2016-09-11 19:35:07,ДТП на Новоприозерском шоссе 11.09.2016\nК нов...,шоссе Новоприозерском 11
58095,4804611,2016-09-11 10:29:17,Угнан автомобиль Тойота Хайлендр 2014 г серебр...,район Приморский улица Сизова Испытателей 10
